# Python para finanças - alocação e otimização de portfólios

### Importação das bibliotecas e base de dados

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.express as px 

In [2]:
dataset = pd.read_csv('acoes2.csv')
dataset

,Date,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,2015-01-02,14.990000,15.20,5.923076,0.232812,11.910702,47.259998
1,2015-01-05,14.850000,15.00,5.963461,0.237187,11.544731,46.320000
2,2015-01-06,15.210000,14.80,5.875000,0.234062,10.822770,46.580002
3,2015-01-07,14.550000,14.67,5.807692,0.241875,10.746248,48.150002
4,2015-01-08,14.270000,14.15,5.905769,0.240000,10.995774,48.509998
...,...,...,...,...,...,...,...
1448,2020-10-27,18.600000,14.17,41.619999,25.450001,28.500000,95.680000
1449,2020-10-28,16.920000,12.77,39.570000,24.629999,27.500000,91.559998
1450,2020-10-29,16.610001,12.68,40.014999,25.360001,28.500000,92.800003
1451,2020-10-30,15.690000,12.28,37.915001,24.629999,27.000000,90.660004


### Alocação aleatória de ativos

In [3]:
dataset.columns  # Mostra as colunas do DataFrame...

Index(['Date', 'GOL', 'CVC', 'WEGE', 'MGLU', 'TOTS', 'BOVA'], dtype='object')

In [4]:
dataset.loc[len(dataset) - 1 ]['BOVA']

92.26000213623048

In [5]:
def alocacao_ativos(dataset, dinheiro_total, seed=0):
    dataset = dataset.copy()

    if seed != 0:
        np.random.seed(seed)
    
    pesos = np.random.random(len(dataset.columns) - 1)  # len() -> passa a quantidade de colunas, menos uma.
    # print(pesos, pesos.sum()) -> valor maior que 1.0
    pesos = pesos / pesos.sum() 
    # print(pesos, pesos.sum())

    # Normalizando o DataFrame: 'dataset'
    colunas = dataset.columns[1:]
    for i in colunas:
        dataset[i] = (dataset[i] / dataset[i][0])

    for i, acao in enumerate(dataset.columns[1:]):
        """
        O enumerate é responsável em gerar os indices, que neste caso
        são de 0 até 5 para que se possa acessar cada uma das posições 
        do vetor: 'pesos'.

        Args:
            i, acao in enumerate (_type_): _description_
            (i) é o índice e (acao) são os valores de cada ação do DataFrame.
            argumento: 'dataset.columns[1:]' são as colunas do DataFrame: 'dataset'.
        """
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    # Adiciona uma coluna efetuando o somatório total das ações em cada data
    dataset['Soma valor'] = dataset[colunas].sum(axis=1)

    datas = dataset['Date']
    # print(datas)
    
    dataset.drop(labels= ['Date'], axis= 1, inplace= True)
    dataset['Taxa retorno'] = 0.0

    # Efetua o calculo da taxa de retorno simples e adiciona o resultado em uma nova coluna
    for i in range(1, len(dataset)):
        dataset['Taxa retorno'][i] = ((dataset['Soma valor'][i] / dataset['Soma valor'][i - 1]) - 1) * 100
    
    acoes_pesos = pd.DataFrame(data = {'Ações': colunas, 'Pesos': pesos * 100})

    return dataset, datas, acoes_pesos, dataset.loc[len(dataset) - 1]['Soma valor']

In [6]:
# Chamando a função
dataset, datas, acoes_pesos, soma_valor = alocacao_ativos(pd.read_csv('acoes2.csv'), 5000, 10)

In [7]:
dataset

,GOL,CVC,WEGE,MGLU,TOTS,BOVA,Soma valor,Taxa retorno
0,1330.859777,35.806036,1093.315674,1292.008683,860.138954,387.870876,5000.000000,0.000000
1,1318.430187,35.334904,1100.770128,1316.288069,833.710152,380.156147,4984.689588,-0.306208
2,1350.392100,34.863773,1084.441499,1298.945650,781.573275,382.290029,4932.506325,-1.046871
3,1291.795217,34.557537,1072.017409,1342.304468,776.047198,395.175284,4911.897113,-0.417824
4,1266.935952,33.332592,1090.120996,1331.899017,794.066880,398.129839,4914.485276,0.052692
...,...,...,...,...,...,...,...,...
1448,1651.367094,33.379707,7682.460263,141236.798956,2058.145750,785.262100,153447.413870,-0.061512
1449,1502.211333,30.081783,7304.059540,136686.134989,1985.930110,751.448534,148259.866289,-3.380668
1450,1474.688596,29.869773,7386.200159,140737.335967,2058.145750,761.625471,152447.865716,2.824769
1451,1393.007983,28.927508,6998.570287,136686.134989,1949.822290,744.062131,147800.525188,-3.048479


In [8]:
datas

0       2015-01-02
1       2015-01-05
2       2015-01-06
3       2015-01-07
4       2015-01-08
           ...    
1448    2020-10-27
1449    2020-10-28
1450    2020-10-29
1451    2020-10-30
1452    2020-11-03
Name: Date, Length: 1453, dtype: object

In [9]:
acoes_pesos

,Ações,Pesos
0,GOL,26.617196
1,CVC,0.716121
2,WEGE,21.866313
3,MGLU,25.840174
4,TOTS,17.202779
5,BOVA,7.757418


In [10]:
soma_valor

149991.6965915256

_____________________________________________________________________________________________________________________________________________________________________

### Visualização dos dados

In [12]:
figura = px.line(x= datas, y= dataset['Taxa retorno'], title= 'Retorno diário do portfólio')
figura.show()

> Obs.: Os valores acima são as taxas de retorno diárias, considerando todo nosso portfólio, inclusive os pesos que foram alocados em cada uma das ações.
_____________________________________________________________________________________________________________________________________________________________________

In [15]:
figura = px.line(title= 'Evolução do patrimônio')
# Loop para adicionar as colunas do dataset sem as colunas: 'soma valor' e 'taxa retorno' 
for i in dataset.drop(columns= ['Soma valor', 'Taxa retorno']).columns:
    figura.add_scatter(x= datas, y= dataset[i], name= i)
figura.show()

In [17]:
figura = px.line(x= datas, y= dataset['Soma valor'], title= 'Evolução do Patrimônio')
figura.show()

> Obs.: O gráfico acima mostra o retorno da carteira no periodo sem demonstrar as ações individuais! 

## Mais calculo no portfólio

### Retorno acumulado em todo período

In [20]:
dataset.loc[len(dataset) - 1]['Soma valor'] / dataset.loc[0]['Soma valor'] - 1

28.99833931830512

### Desvio padrão

In [21]:
dataset['Taxa retorno'].std()
# Quanto maior o valor, maior é o risco da carteira de investimentos
#pelo fato de existir uma variabilidade maior... 

2.8857783839149223

### Sharpe ratio médio anual, sem considerar a taxa SELIC

In [22]:
(dataset['Taxa retorno'].mean() / dataset['Taxa retorno'].std()) * np.sqrt(246)

1.5003826486511682

> Obs.: Quanto maior for o valor do 'Sharpe Ratio' melhor:

- acima de 1.0: é considerado aceitável;
- acima de 2.0: é considerado bom;
- acima de 3.0: é considerado excelente.

> Para encontrar o valor do 'risk free' é necessário efetuar o calculo abaixo:

In [24]:
dinheiro_total = 5000

In [25]:
soma_valor - dinheiro_total

144991.6965915256

In [26]:
# Taxa selic: https://www.infomoney.com.br/guias/taxa-selic/
taxa_selic_2015 = 12.75
taxa_selic_2016 = 14.25
taxa_selic_2017 = 12.25
taxa_selic_2018 = 6.50
taxa_selic_2019 = 5.00
taxa_selic_2020 = 2.0


> Calculo de taxa de renda fixa do ano de 2015:

In [27]:
# 2015

valor_2015 = dinheiro_total + (dinheiro_total * taxa_selic_2015 / 100)
valor_2015
# Caso tenha investido R$ 5000.00 em um ano no final teria o valor de R$ 5.634,50
#conforme o resultado abaixo! teria

5637.5

In [29]:
# 2016

valor_2016 = valor_2015 + (valor_2015 * taxa_selic_2016 / 100)
valor_2016

6440.84375

In [30]:
# 2017

valor_2017 = valor_2016 + (valor_2016 * taxa_selic_2017 / 100)
valor_2017

7229.8471093749995

In [31]:
# 2018

valor_2018 = valor_2017 + (valor_2017 * taxa_selic_2018 / 100)
valor_2018

7699.787171484374

In [32]:
# 2019

valor_2019 = valor_2018 + (valor_2018 * taxa_selic_2019 / 100)
valor_2019

8084.7765300585925

In [33]:
# 2020 - valor bruto:

valor_2020 = valor_2019 + (valor_2019 * taxa_selic_2020 / 100)
valor_2020

8246.472060659764

In [42]:
taxa_selic_historico = np.array([12.75, 14.25, 12.25, 6.5, 5.0, 2.0])
taxa_selic_historico.mean() / 100

0.08791666666666666

In [44]:
# Valor em percentual do Sharpe Ratio considerando a média da taxa SELIC
(dataset['Taxa retorno'].mean() - taxa_selic_historico.mean() / 100) / dataset['Taxa retorno'].std() * np.sqrt(246)

1.0225500320290082

>Rendimento adquiridos no investimento de renda fixa:

In [34]:
rendimentos = valor_2020 - dinheiro_total
rendimentos

3246.472060659764

In [35]:
# Desconto do Imposto de Renda:
ir = rendimentos * 15 /100
ir

486.9708090989646

In [36]:
# Valor liquido total:
valor_2020 - ir

7759.5012515608